In [1]:
import numpy as np
import numpy.linalg as la
from unyt import accepts, returns
import unyt as u
import unyt.dimensions as ud

from rocketPy import Quaternion

In [2]:
class Rocket():
    
    def __init__(self, mass=40*u.kg):
        
        #this calls on the setter function
        self.m = mass
        
        self.theta = 6*u.degree
        
        self.inertia_matrix = np.diag([100,100,10])*u.kg/u.m**2

        
    # this is all extra code to handle getting and setting
    @property
    def m(self): return self._m*u.kg
    
    @m.setter
    @accepts(value=ud.mass) #checks that when setting, a correct dimensional value is provided
    def m(self, value): self._m = value.in_base().v
    
    @property
    def theta(self): return self._theta*u.radian
    
    @theta.setter
    @accepts(value=ud.angle)
    def theta(self, value): self._theta = value.in_base().v
    
    @property
    def inertia_matrix(self): return self._inertia_matrix * u.kg/u.m**2
    
    @inertia_matrix.setter
    @accepts(value=ud.mass/ud.length**2)
    def inertia_matrix(self, value): self._inertia_matrix = value.in_base().v
    

In [3]:
# define values with units at creation of the object
r = Rocket(mass=35*u.kg)

In [4]:
# for a user, getting the value is straightforward, and is returned with units
r.m

unyt_quantity(35., 'kg')

In [5]:
# when setting for the user, it has to be with units
r.m = 14*u.kg

In [6]:
# again, getting the value is in units
r.m

unyt_quantity(14., 'kg')

In [7]:
# but a developer can access the si version directly
r._m

array(14.)

In [8]:
# modify it
r._m = 99

In [9]:
# and the user will see the modified version
r.m

unyt_quantity(99, 'kg')

In [10]:
# similarly, all angles are automatically converted into radians
r.theta

unyt_quantity(0.10471976, 'rad')

In [11]:
r._theta

array(0.10471976)

In [12]:
# notice that internally only si is used
r.__dict__

{'_m': 99,
 '_theta': array(0.10471976),
 '_inertia_matrix': array([[100.,   0.,   0.],
        [  0., 100.,   0.],
        [  0.,   0.,  10.]])}

In [13]:
# this is where the getters and setters are really important
# unyt doesnt respect many linear algebra commands
# for instance the inertia matrix has untis kg/m^2
r.inertia_matrix

unyt_array([[100.,   0.,   0.],
            [  0., 100.,   0.],
            [  0.,   0.,  10.]], 'kg/m**2')

In [14]:
# if we access the si version directly
r._inertia_matrix

array([[100.,   0.,   0.],
       [  0., 100.,   0.],
       [  0.,   0.,  10.]])

In [15]:
# now if we invert the matrix, the units are still kg/m^2 not m^2/kg
la.inv(r.inertia_matrix)

unyt_array([[0.01, 0.  , 0.  ],
            [0.  , 0.01, 0.  ],
            [0.  , 0.  , 0.1 ]], 'kg/m**2')

In [16]:
# but the 'si' version would be (to the developer) interpretted correctly
la.inv(r._inertia_matrix)

array([[0.01, 0.  , 0.  ],
       [0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.1 ]])

In [17]:
# furthermore this approach should make simulations much faster
# the developer must be careful about this